## Pytorch Text Classiciation from CSV

code example from https://www.kaggle.com/code/swarnabha/pytorch-text-classification-torchtext-lstm

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time

import torch
from torch.utils.data import DataLoader
from torchtext import data
import torch.nn as nn

#### Import data

Data is found from kaggles nlp getting starting datatset

In [2]:
test = pd.read_csv("/Users/eric/Documents/GitHub/pytorch-example/text/data/nlp-getting-started/test.csv")
train = pd.read_csv("/Users/eric/Documents/GitHub/pytorch-example/text/data/nlp-getting-started/train.csv")

#### Data Pre Processing

Review and cleaning the text data

In [3]:
train.shape

(7613, 5)

In [4]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


The target columns marks the label of each of the text
- label = 1 is the tweet is about disasters
- label = 0 is the tweet is not about disasters

In [5]:
train.drop(columns=['id', 'keyword', 'location'], inplace=True)

##### Text cleaning

Clean up text dataa for the classification algorithm does not get confused with incorrect data

In [6]:
def normalise_text (text):
    text = text.str.lower() # lowercase
    text = text.str.replace("#", "") # removes hastags
    text = text.str.replace("http\S+","URL") # remove url address
    text = text.str.replace("@", "")
    text = text.str.replace("[^A-Za-z0-9()!?\'\`\"]", " ")
    text = text.str.replace("\s{2,}", " ")
    return text

In [7]:
train["text"].head()

0    Our Deeds are the Reason of this #earthquake M...
1               Forest fire near La Ronge Sask. Canada
2    All residents asked to 'shelter in place' are ...
3    13,000 people receive #wildfires evacuation or...
4    Just got sent this photo from Ruby #Alaska as ...
Name: text, dtype: object

In [8]:
train["text"] = normalise_text(train['text'])

In [9]:
train["text"].head()

0    our deeds are the reason of this earthquake ma...
1               forest fire near la ronge sask. canada
2    all residents asked to 'shelter in place' are ...
3    13,000 people receive wildfires evacuation ord...
4    just got sent this photo from ruby alaska as s...
Name: text, dtype: object

split the data into training and validation 

In [10]:
train_df, valid_df = train_test_split(train)

In [11]:
train_df.head()

,text,target
716,now my nose is bleeding. the last one was like...,0
1455,casualty roleplay somebody please am so bored,0
7279,whirlwind head scissor on alexhammerstone ktta...,0
1859,master0fsloths has a crush: http://t.co/szx6v0...,0
1880,do he love me do he love me not i ain't a play...,0


In [12]:
valid_df.head()

,text,target
3694,i added a video to a youtube playlist http://t...,0
668,i liked a youtube video http://t.co/n95igskd3p...,0
1430,property/casualty insurance rates up 1% in jul...,1
3776,fire call: brant av / drummond rd for fire - *...,1
6782,rly tragedy in mp: some live to recount horror...,1


#### Set repoducibility

setup tourch to create reproducibility from machine to machine and run to run

In [13]:
SEED = 42

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

##### Preprocessing setup

In [17]:
def collate_fn(batch):
    texts, labels = [], []
    for label, txt in batch:
        texts.append(txt)
        labels.append(label)
    return texts, labels

In [ ]:
dataloader = DataLoader(train)